In [1]:
import pandas as pd
import win32com.client as win32
import time
import requests
import json

Abrindo Data Frames

Puxar Valor Dolar

In [2]:
requisicao = requests.get(r'https://economia.awesomeapi.com.br/all/USD-BRL')
cotacao = requisicao.json()
cotacao_dolar = float(cotacao['USD']['bid'])
print(cotacao_dolar)

5.4009


Excel file, open, refresh and close

In [3]:
# Start an instance of Excel
xlapp = win32.DispatchEx("Excel.Application")

# Open the workbook in said instance of Excel
wb = xlapp.workbooks.open(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')

# Optional, e.g. if you want to debug
xlapp.Visible = True

# Refresh all data connections.
wb.RefreshAll()
wb.Save()

# Quit
xlapp.Quit()

In [4]:
base_dash_df = pd.read_excel(
    r'base_dash.xlsx', index_col=[0])
infos_df = pd.read_excel(
    r'infos.xlsx', index_col=[0])
display(base_dash_df)
display(infos_df)


,Data,Fundos de Investimentos,Fundos Imobiliários,Ações,COE,Renda Fixa,Disponível,Cripto,Dow Jones,Total
NaN,20/07/2022,117484.14,487055.63,14198.81,5000,13415.65,13175.88,4407.69,68900.0000,723637.8000
NaN,27/07/2022,122779.91,533358.48,13490.60,5000,13416.94,1096.02,4407.69,68900.0000,762449.6400
NaN,28/07/2022,122899.81,533217.58,14025.40,5000,13414.85,1096.02,4407.69,68900.0000,762961.3500
NaN,08/08/2022,123428.76,535534.83,14904.06,5000,13400.25,2591.90,4407.69,68900.0000,765575.5900
NaN,09/08/2022,123489.84,538180.19,14763.26,5000,13398.16,2591.90,4407.69,68900.0000,768139.1400
NaN,11/08/2022,123615.01,535089.68,14398.16,5000,13392.04,2591.90,4842.39,90577.6500,786480.2300
NaN,13/08/2022,123678.97,540386.52,14431.89,5000,13389.83,5706.41,4842.39,90577.6500,792307.2500
NaN,16/08/2022,123742.97,542476.77,14873.15,5000,13385.41,5706.41,4842.39,90577.6500,794898.3400
NaN,17/08/2022,123809.72,543304.23,14836.99,5000,13383.20,6958.82,4965.27,92513.2500,797812.6600
NaN,18/08/2022,123939.31,545387.46,14860.73,5000,13382.43,6958.82,4823.18,94364.0165,808715.9465


,Data,Fundos de Investimentos,Fundos Imobiliários,Ações,COE,Renda Fixa,Disponível,Cripto,Dow Jones,Total
NaN,29/09/2022,132858.73,548647.73,12529.73,5000,13374.21,0,4127.2,83409.9048,799947.5048


Concatenar com novas infos

In [5]:
bases_df = [base_dash_df, infos_df]
base_dash_df = pd.concat(bases_df, )
# base_dash_df.iloc[-1]['Dow Jones'] = base_dash_df.iloc[-1]['Dow Jones'] * cotacao_dolar
# print(base_dash_df.iloc[-1]['Dow Jones'])
print(base_dash_df)
base_dash_df.to_excel(r'base_dash.xlsx')


           Data  Fundos de Investimentos  Fundos Imobiliários     Ações   COE  \
NaN  20/07/2022                117484.14            487055.63  14198.81  5000   
NaN  27/07/2022                122779.91            533358.48  13490.60  5000   
NaN  28/07/2022                122899.81            533217.58  14025.40  5000   
NaN  08/08/2022                123428.76            535534.83  14904.06  5000   
NaN  09/08/2022                123489.84            538180.19  14763.26  5000   
NaN  11/08/2022                123615.01            535089.68  14398.16  5000   
NaN  13/08/2022                123678.97            540386.52  14431.89  5000   
NaN  16/08/2022                123742.97            542476.77  14873.15  5000   
NaN  17/08/2022                123809.72            543304.23  14836.99  5000   
NaN  18/08/2022                123939.31            545387.46  14860.73  5000   
NaN  19/08/2022                124004.04            544898.73  14289.14  5000   
NaN  22/08/2022             

Plotar Gráfico

#base_dash_df.plot(x="Data", figsize=(20, 10))
#display(base_dash_df)

Procurando data do Posição Detalhada

In [7]:
posicao_detalhada_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\PosicaoDetalhada.xlsx', index_col=[0])
posicao = posicao_detalhada_df.columns[4]
data_posicao_detalhada = posicao[17:27]
print(data_posicao_detalhada)


29/09/2022


Enviando por email

'''outlook = win32.Dispatch('outlook.application')

data = data_posicao_detalhada
email = 'beligolli@hotmail.com'


mail = outlook.CreateItem(0)
mail.To = email
mail.Subject = 'Dash_Financeiro - {}.'.format(data)
mail.Body = """
Segue Dash Financeiro até o dia {} 

att.,
""".format(data)
#ou mail.HTMLBody = '<p>Corpo do Email em HTML</p>'

# Anexos (pode colocar quantos quiser):
attachment = r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\base_dash.xlsx'
mail.Attachments.Add(attachment)

mail.Send()'''

Codigo para informar que a operação foi concluida

In [ ]:
# from tkinter import *
# from tkinter import messagebox
# janela = Tk()
# messagebox.showinfo('Status', 'Processo Concluído.')
# janela.destroy